In [33]:
import pandas as pd
import numpy as np
import psycopg2 as psy
import query_to_redshift as qtr

In [34]:
original_df_up = pd.read_csv('../raw_data/Excel_files/Item_margin_up_pre_pc2_change_Feb2021.csv', index_col=0)
original_df_down = pd.read_csv('../raw_data/Excel_files/Item_margin_down_pre_pc2_change_Feb2021.csv', index_col=0)

In [35]:
homie_days_query = """
SELECT item_code,
       MIN(period_date),
       MAX(period_date),
       webshop_code,
       final_price_amount,
       item_campaign_key,
       item_campaign_label,
       item_campaign_category
  from public.item_catalog_history
 where item_campaign_category like '%HomieDay%'
   and item_campaign_key like '%14%'
   and sellable_flag = true
   and webshop_code = 'DE'
   and period_date >= '2021-04-29'
 GROUP BY item_code, webshop_code, final_price_amount,
 item_campaign_label, item_campaign_key, item_campaign_category
HAVING (MAX(period_date) - MIN(period_date)) > 14."""


In [36]:
homie_bazooka_query = """
SELECT item_code,
       MIN(period_date) as min_date,
       MAX(period_date) as max_date,
       --webshop_code,
       MAX(final_price_amount) as max_price,
       MAX(item_campaign_key) as max_item_campaign_key,
       MAX(item_campaign_label) as max_item_campaign_label,
       MAX(item_campaign_category) as max_item_campaign_category
  from public.item_catalog_history
 where item_campaign_category like '[CAMPAIGN]HomieDay_Voucher_CW17|%'
   and item_campaign_key like 'party14%'
   and sellable_flag = true
   and webshop_code = 'DE'
   and period_date >= '2021-04-29'
 GROUP BY item_code
HAVING MIN(period_date) = '2021-04-29'
   and MAX(period_date) = '2021-05-10';"""


In [37]:
#df_homie14 = qtr.redshift_query_to_dataframe(qtr.cursor_redshift, homie_days_query)
df_homie14 = qtr.redshift_query_to_dataframe(qtr.cursor_redshift, homie_bazooka_query)

In [38]:
print(len(df_homie14.index))

7335


In [39]:
print(df_homie14.head())

            item_code    min_date    max_date                max_price  \
0  000000001000102199  2021-04-29  2021-05-10   449.990000000000000000   
1  000000001000239023  2021-04-29  2021-05-10    66.990000000000000000   
2  000000001000011689  2021-04-29  2021-05-10  1249.990000000000000000   
3  000000001000225158  2021-04-29  2021-05-10   379.990000000000000000   
4  000000001000144202  2021-04-29  2021-05-10    79.990000000000000000   

  max_item_campaign_key  max_item_campaign_label  \
0   party14|testcynthia  EXTRA -14 %|testcynthia   
1        party14|30sale           EXTRA -14 %|30   
2      party14|24series     EXTRA -14 %|EXKLUSIV   
3        party14|30sale           EXTRA -14 %|30   
4        party14|30sale           EXTRA -14 %|30   

         max_item_campaign_category  
0  [CAMPAIGN]HomieDay_Voucher_CW17|  
1  [CAMPAIGN]HomieDay_Voucher_CW17|  
2  [CAMPAIGN]HomieDay_Voucher_CW17|  
3  [CAMPAIGN]HomieDay_Voucher_CW17|  
4  [CAMPAIGN]HomieDay_Voucher_CW17|  


In [40]:
df_homie14.dtypes

item_code                     object
min_date                      object
max_date                      object
max_price                     object
max_item_campaign_key         object
max_item_campaign_label       object
max_item_campaign_category    object
dtype: object

In [41]:
df_homie14 = df_homie14.astype({'item_code': int})

In [42]:
df_homie14.dtypes

item_code                      int64
min_date                      object
max_date                      object
max_price                     object
max_item_campaign_key         object
max_item_campaign_label       object
max_item_campaign_category    object
dtype: object

In [43]:
df_homie14.head()


,item_code,min_date,max_date,max_price,max_item_campaign_key,max_item_campaign_label,max_item_campaign_category
0,1000102199,2021-04-29,2021-05-10,449.990000000000000000,party14|testcynthia,EXTRA -14 %|testcynthia,[CAMPAIGN]HomieDay_Voucher_CW17|
1,1000239023,2021-04-29,2021-05-10,66.990000000000000000,party14|30sale,EXTRA -14 %|30,[CAMPAIGN]HomieDay_Voucher_CW17|
2,1000011689,2021-04-29,2021-05-10,1249.990000000000000000,party14|24series,EXTRA -14 %|EXKLUSIV,[CAMPAIGN]HomieDay_Voucher_CW17|
3,1000225158,2021-04-29,2021-05-10,379.990000000000000000,party14|30sale,EXTRA -14 %|30,[CAMPAIGN]HomieDay_Voucher_CW17|
4,1000144202,2021-04-29,2021-05-10,79.990000000000000000,party14|30sale,EXTRA -14 %|30,[CAMPAIGN]HomieDay_Voucher_CW17|


In [44]:
merged_down = pd.merge(original_df_down, df_homie14, on='item_code')


In [45]:
merged_up = pd.merge(original_df_up, df_homie14, on='item_code')

In [46]:
merged_up.head()


,item_code,item_price_x,max_date_x,delivery_weeks,sales_pe,sales_pe_err,beta_del_week,beta_del_week_err,intercept,intercept_err,...,N_prime_sales,Delta_Margin,Group_flag,Increment,min_date,max_date_y,max_price,max_item_campaign_key,max_item_campaign_label,max_item_campaign_category
0,1000008030,229.99,2021-02-22,2,-2.3235,0.4060,-0.0690,0.0178,16.6206,2.8286,...,3.542880,5.786855,up,14.000000000000002%,2021-04-29,2021-05-10,269.990000000000000000,party14|24series|3dmodels|crm-japandiws,EXTRA -14 %|EXKLUSIV|3dmodels|crm,[CAMPAIGN]HomieDay_Voucher_CW17||2019032601|
1,1000118440,74.99,2021-02-25,1,-8.6519,2.6054,-0.3592,0.1629,49.4656,14.4964,...,10.619435,5.160529,up,14.000000000000002%,2021-04-29,2021-05-10,74.990000000000000000,party14|40sale,EXTRA -14 %|40,[CAMPAIGN]HomieDay_Voucher_CW17|
2,1000008014,429.99,2021-02-25,1,-0.6558,0.1648,-0.0129,0.0078,4.7260,0.9924,...,0.650557,1.100490,up,14.000000000000002%,2021-04-29,2021-05-10,449.990000000000000000,party14|24series,EXTRA -14 %|EXKLUSIV,[CAMPAIGN]HomieDay_Voucher_CW17|
3,1000008012,579.99,2021-02-25,2,-2.9931,0.8553,-0.0630,0.0366,21.8786,6.1560,...,2.315291,3.198499,up,14.000000000000002%,2021-04-29,2021-05-10,579.990000000000000000,party14|24series|3dmodels|storage|crm-japandiws,EXTRA -14 %|EXKLUSIV|3dmodels|storage|crm,[CAMPAIGN]HomieDay_Voucher_CW17||2019032601||
4,1000086540,349.99,2021-02-25,2,-1.3911,0.3143,-0.0550,0.0133,11.0650,2.0811,...,2.623796,7.090544,up,14.000000000000002%,2021-04-29,2021-05-10,349.990000000000000000,party14|3dmodels,EXTRA -14 %|3dmodels,[CAMPAIGN]HomieDay_Voucher_CW17|2019032601


In [47]:
merged_down.head()

,item_code,item_price_x,max_date_x,delivery_weeks,sales_pe,sales_pe_err,beta_del_week,beta_del_week_err,intercept,intercept_err,...,N_prime_sales,Delta_Margin,Group_flag,Increment,min_date,max_date_y,max_price,max_item_campaign_key,max_item_campaign_label,max_item_campaign_category
0,1000147156,429.99,2021-02-25,5,-3.2220,0.3670,-0.0276,0.0102,18.8254,2.0617,...,-0.364090,7.237100,down,-14.000000000000002%,2021-04-29,2021-05-10,429.990000000000000000,party14|24series,EXTRA -14 %|EXKLUSIV,[CAMPAIGN]HomieDay_Voucher_CW17|
1,1000048003,349.99,2021-02-25,2,-2.3721,0.5706,0.0010,0.0125,14.1651,3.0970,...,0.629332,0.182455,down,-14.000000000000002%,2021-04-29,2021-05-10,349.990000000000000000,party14|storage,EXTRA -14 %|storage,[CAMPAIGN]HomieDay_Voucher_CW17|
2,1000129691,214.99,2021-01-10,2,-2.4983,0.3057,-0.0395,0.0149,12.4723,1.4595,...,-0.647248,2.043591,down,-14.000000000000002%,2021-04-29,2021-05-10,229.990000000000000000,party14|50sale,EXTRA -14 %|Sale,[CAMPAIGN]HomieDay_Voucher_CW17|
3,1000008520,269.99,2021-02-16,3,-4.3979,0.5157,-0.0934,0.0422,26.1283,3.0246,...,1.890267,0.340697,down,-14.000000000000002%,2021-04-29,2021-05-10,269.990000000000000000,party14|crm-tvspot,EXTRA -14 %|crm,[CAMPAIGN]HomieDay_Voucher_CW17|
4,1000128456,429.99,2021-02-08,6,-1.3928,0.3220,-0.0133,0.0162,8.3261,1.8308,...,0.010758,1.906100,down,-14.000000000000002%,2021-04-29,2021-05-10,429.990000000000000000,party14|eurocup|eurocup21|24series,EXTRA -14 %|Euro Cup|Euro Cup|EXKLUSIV,[CAMPAIGN]HomieDay_Voucher_CW17|||


In [48]:
list_of_items_up = merged_up['item_code'].to_numpy()

In [49]:
list_of_items_down = merged_down['item_code'].to_numpy()


In [50]:

print(list_of_items_up)

[1000008030 1000118440 1000008014 1000008012 1000086540 1000123399
 1000152766 1000008000 1000177730 1000209641 1000184191 1000124029
 1000002479 1000086500 1000209643 1000008842 1000056963 1000092019
 1000181275 1000104255 1000031300 1000092055 1000193120 1000121202
 1000027408 1000008016 1000150226 1000189778 1000136114 1000153429
 1000086536 1000103647 1000164014 1000205104 1000037843 1000207389
 1000189771 1000073217 1000209634 1000008032 1000117190 1000012529
 1000119257 1000008031 1000008033 1000008872 1000050844 1000012567
 1000039246 1000189796 1000012678 1000015909 1000202053 1000008824
 1000110022 1000033524 1000216583 1000076797 1000153434 1000086499
 1000123195 1000008037 1000008039 1000012562 1000129379 1000008754
 1000119769 1000008035 1000092325 1000214952 1000123395 1000126299]


In [51]:
print(list_of_items_down)

[1000147156 1000048003 1000129691 1000008520 1000128456 1000039240
 1000110349 1000176687 1000123763 1000056781 1000015037 1000064653
 1000110348 1000056742 1000087664 1000008825 1000024953 1000007069
 1000086542 1000056777 1000007050 1000117189 1000007079 1000008790
 1000008056 1000050883 1000008867 1000125821 1000008752 1000027368
 1000176894 1000056779 1000110431 1000202440 1000112304 1000113650
 1000148080 1000012676 1000126227 1000158712 1000092554 1000007429
 1000008058 1000007430 1000172230 1000112303 1000005213 1000197490
 1000007080 1000008833 1000022436 1000168548 1000056747 1000131873
 1000113296 1000039128 1000007433 1000008748 1000008007 1000104040
 1000126218 1000064654 1000125642 1000008755 1000034491 1000007107
 1000050873 1000056975 1000056783 1000008814 1000210654 1000139406
 1000008815 1000007083 1000091560 1000027357 1000210651 1000008067
 1000024284 1000206722 1000008807 1000177736 1000008809 1000050870
 1000210650 1000022466 1000209631 1000129637 1000126687 100012

In [52]:
merged_up['item_code'].to_csv('../raw_data/Excel_files/list_items_up_prePc2_bazooka.csv')
merged_down['item_code'].to_csv('../raw_data/Excel_files/list_items_down_prePc2_bazooka.csv')